In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
import re

def load_jsonl(file_path):
    """Load data from JSONL file"""
    with open(file_path, 'r', encoding="utf-8") as file:
        return [json.loads(line) for line in file]

def diplomacy_tokenizer(text):
    """Custom tokenizer for Diplomacy game messages"""
    text = text.lower()
    
    # Handle emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' emoji ', text)
    
    # Handle territory names
    territory_names = ['marseille', 'iberia', 'brest', 'paris', 'tyrolia', 'trieste', 'burgundy', 
                      'mediterranean', 'adriatic', 'aegean', 'english channel', 'baltic', 'north sea',
                      'western mediterranean', 'eastern mediterranean', 'ionian sea', 'tyrrhenian sea']
    for territory in territory_names:
        text = text.replace(territory, f"TERRITORY_{territory.replace(' ', '_')}")
    
    # Handle game terms
    game_terms = ['attack', 'defend', 'support', 'convoy', 'build', 'retreat', 'disband', 
                 'hold', 'move', 'bounce', 'cut', 'dislodge', 'alliance', 'truce']
    for term in game_terms:
        text = text.replace(term, f"GAMETERM_{term}")
    
    return re.findall(r'\b\w+\b', text)

def extract_harbinger_features(message):
    """Extract linguistic features from messages"""
    message_lower = message.lower()
    
    features = {
        # Politeness
        'has_please': 'please' in message_lower,
        'has_thanks': any(word in message_lower for word in ['thanks', 'thank you', 'appreciate']),
        'has_sorry': any(word in message_lower for word in ['sorry', 'apologies', 'apologize', 'my bad']),
        
        # Alignment
        'mentions_we': any(word in message_lower.split() for word in ['we', 'us', 'our', 'together', 'both']),
        'mentions_you': any(word in message_lower.split() for word in ['you', 'your', 'yours']),
        'mentions_i': any(word in message_lower.split() for word in ['i', 'me', 'my', 'mine']),
        
        # Engagement
        'asks_question': '?' in message,
        'uses_exclamation': '!' in message,
        'message_length': len(message),
        'avg_word_length': np.mean([len(word) for word in message_lower.split()]) if message_lower.split() else 0,
        
        # Emotion/Certainty
        'has_emoji': bool(re.search(r'[\U0001F000-\U0001FFFF]', message)),
        'uses_hedging': any(word in message_lower for word in ['maybe', 'perhaps', 'possibly', 'might', 'could', 'would', 'should']),
        'uses_certainty': any(word in message_lower for word in ['definitely', 'certainly', 'absolutely', 'surely', 'will', 'must']),
        'expresses_positive': any(word in message_lower for word in ['good', 'great', 'excellent', 'awesome', 'fantastic', 'wonderful', 'perfect', 'happy']),
        'expresses_negative': any(word in message_lower for word in ['bad', 'terrible', 'awful', 'unfortunate', 'sad', 'unhappy', 'sorry', 'worry']),
        
        # Strategy/Deception
        'mentions_attack': any(word in message_lower for word in ['attack', 'invade', 'move', 'capture', 'take']),
        'mentions_defense': any(word in message_lower for word in ['defend', 'hold', 'protect', 'support', 'cover']),
        'mentions_alliance': any(word in message_lower for word in ['ally', 'alliance', 'together', 'support', 'help', 'cooperate', 'friend']),
        'mentions_betrayal': any(word in message_lower for word in ['betray', 'lie', 'cheat', 'backstab', 'break']),
        'future_planning': any(word in message_lower for word in ['plan', 'future', 'next', 'later', 'after', 'then', 'will']),
        'offers_deal': any(word in message_lower for word in ['deal', 'offer', 'agreement', 'compromise', 'exchange']),
        'uses_justification': any(word in message_lower for word in ['because', 'since', 'reason', 'explain', 'understand']),
        'expresses_concern': any(word in message_lower for word in ['concern', 'worried', 'scared', 'afraid', 'fear']),
        
        # Game references
        'mentions_territory': any(word in message_lower for word in ['marseille', 'iberia', 'brest', 'paris', 'tyrolia', 'trieste', 'burgundy', 'mediterranean']),
        'mentions_country': any(word in message_lower for word in ['france', 'germany', 'italy', 'england', 'austria', 'russia', 'turkey']),
        'mentions_season': any(word in message_lower for word in ['spring', 'fall', 'winter', 'year']),
        'mentions_unit': any(word in message_lower for word in ['fleet', 'army', 'unit', 'forces', 'troops']),
    }
    
    return features

def preprocess_diplomacy_data(data):
    """Process raw game data into features and labels"""
    all_messages = []
    all_features = []
    all_labels = []
    
    for game in data:
        messages = game.get("messages", [])
        sender_labels = game.get("sender_labels", [])
        receiver_labels = game.get("receiver_labels", [])
        speakers = game.get("speakers", [])
        receivers = game.get("receivers", [])
        seasons = game.get("seasons", [])
        years = game.get("years", [])
        game_score = game.get("game_score", [])
        game_score_delta = game.get("game_score_delta", [])
        game_id = game.get("game_id", "")
        
        if not messages or len(messages) < 2:
            continue
        
        for i in range(len(messages) - 1):
            message = messages[i]
            
            base_features = {
                'game_id': game_id,
                'sender': speakers[i] if i < len(speakers) else "",
                'receiver': receivers[i] if i < len(receivers) else "",
                'season': seasons[i] if i < len(seasons) else "",
                'year': years[i] if i < len(years) else "",
                'current_score': int(game_score[i]) if i < len(game_score) else 0,
                'is_sender_player': sender_labels[i] if i < len(sender_labels) else False,
                'is_receiver_player': receiver_labels[i] if i < len(receiver_labels) else False,
                'message_index': i,
                'total_messages': len(messages),
                'relative_position': i / len(messages) if messages else 0,
            }
            
            next_score_delta = int(game_score_delta[i+1]) if i+1 < len(game_score_delta) else 0
            harbinger_features = extract_harbinger_features(message)
            combined_features = {**base_features, **harbinger_features}
            label = 1 if next_score_delta < 0 else 0
            
            all_messages.append(message)
            all_features.append(combined_features)
            all_labels.append(label)
    
    return all_messages, pd.DataFrame(all_features), all_labels

def print_metrics(true_labels, predictions, probabilities, set_name):
    """Print evaluation metrics for a dataset"""
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    auc = roc_auc_score(true_labels, probabilities)
    
    print(f"\n{set_name} Set Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")
    
    report = classification_report(true_labels, predictions, output_dict=True)
    print(f"F1 Score (Truth Class): {report['0']['f1-score']:.4f}")
    print(f"F1 Score (Lie Class): {report['1']['f1-score']:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(true_labels, predictions))

def main():
    # Load datasets
    print("Loading datasets...")
    train_data = load_jsonl(r"C:\Users\pratham\Downloads\train.jsonl")
    val_data = load_jsonl(r"C:\Users\pratham\Downloads\validation.jsonl")
    test_data = load_jsonl(r"C:\Users\pratham\Downloads\test.jsonl")
    
    # Preprocess data
    print("\nPreprocessing data...")
    train_messages, train_features, train_labels = preprocess_diplomacy_data(train_data)
    val_messages, val_features, val_labels = preprocess_diplomacy_data(val_data)
    test_messages, test_features, test_labels = preprocess_diplomacy_data(test_data)
    
    print(f"\nDataset sizes:")
    print(f"Training set: {len(train_messages)} samples")
    print(f"Validation set: {len(val_messages)} samples")
    print(f"Test set: {len(test_messages)} samples")
    
    # Prepare data
    X_train = pd.concat([train_features, pd.Series(train_messages, name='message')], axis=1)
    X_val = pd.concat([val_features, pd.Series(val_messages, name='message')], axis=1)
    X_test = pd.concat([test_features, pd.Series(test_messages, name='message')], axis=1)
    y_train, y_val, y_test = train_labels, val_labels, test_labels
    
    # Define preprocessing
    categorical_features = ['sender', 'receiver', 'season', 'year']
    numeric_features = [col for col in train_features.columns if train_features[col].dtype in [np.int64, np.float64]]
    boolean_features = [col for col in train_features.columns if train_features[col].dtype == bool]
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
            ('num', StandardScaler(), numeric_features),
            ('bool', 'passthrough', boolean_features),
            ('txt', TfidfVectorizer(
                max_features=500, 
                ngram_range=(1, 3),
                tokenizer=diplomacy_tokenizer,
                min_df=3
            ), 'message')
        ],
        remainder='drop'
    )
    
    # Build pipeline
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            C=1.0,
            class_weight='balanced',
            max_iter=1000,
            random_state=42,
            solver='liblinear'
        ))
    ])
    
    # Train model
    print("\nTraining model...")
    model.fit(X_train, y_train)
    
    # Evaluate
    for X, y, name in [(X_val, y_val, "Validation"), (X_test, y_test, "Test")]:
        preds = model.predict(X)
        probs = model.predict_proba(X)[:, 1]
        print_metrics(y, preds, probs, name)

if __name__ == "__main__":
    main()

Loading datasets...

Preprocessing data...

Dataset sizes:
Training set: 12948 samples
Validation set: 1396 samples
Test set: 2699 samples

Training model...


C:\Users\pratham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



Validation Set Performance:
Accuracy: 0.7142
F1 Score: 0.3677
AUC-ROC: 0.6715
F1 Score (Truth Class): 0.8154
F1 Score (Lie Class): 0.3677

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.87      0.82      1008
           1       0.48      0.30      0.37       388

    accuracy                           0.71      1396
   macro avg       0.62      0.59      0.59      1396
weighted avg       0.68      0.71      0.69      1396


Test Set Performance:
Accuracy: 0.5517
F1 Score: 0.4988
AUC-ROC: 0.6323
F1 Score (Truth Class): 0.5945
F1 Score (Lie Class): 0.4988

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.48      0.59      1865
           1       0.38      0.72      0.50       834

    accuracy                           0.55      2699
   macro avg       0.59      0.60      0.55      2699
weighted avg       0.67      0.55      0.56      2699

